# Knapsack Problem

The knapsack problem is a common problem in combinatorial optimization. Our objective in such problems, given a set of n items (each with a weight and a value), is to find the number of each item to include in a collection such that the total weight is less than or equal to a given limit and the total value is as large as possible. 

In simple terms, we want to gather the most valuable items without overloading our knapsack.

The two popular variants of this problem are:
1. 0/1 knapsack problem
2. Continous/fractional knapsack problem

This post implements 1. 

Mathmatically: 

Maximize: 
$$\sum\limits_{n=0}^{n-1} V_i*X_i$$

Subject to: 
$$\sum\limits_{n=0}^{n-1} W_i*X_i <= W$$

where, X = item in {0,1} 

       W = weight
       
       V = value

For this particular example consider that you are going on a short trip and can only carry a certain number of items among: book, pen, pencil, ipad, iphone, headphones, flask, food and a portable charger with a total value of 504.

Now the question remains, what combination of these items will give you the higest bang for your buck? or in this case knapsack.

# Approach1: Greedy algorithm

Step1: Create a class Knap which has functions for value, cost and density

In [1]:
#create a class for Knapsack
class Knap(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.weight = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.weight
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.weight) + '>'

Step2: Create a function for inputs

In [2]:
def buildInput(names, values, weight):
    '''Accepts 3 different lists and returns a list'''
    Input = []
    for i in range(len(values)):
        Input.append(Knap(names[i], values[i],
                          weight[i]))
    return Input


Create a function to perform greedy optimization

In [3]:
#creating the greedy function
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of Items to numbers"""
    itemsCopy = sorted(items, key = keyFunction,
                       reverse = True)
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)


In [4]:
#Create a function to use greedy
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

In [5]:
#using greedy on 3 different use cases
def testGreedys(inp, maxUnits):
    '''Applies the testGreedy function for three use cases
        value, cost, density'''
    print('Use greedy by value to allocate a weight of', maxUnits)
    testGreedy(inp, maxUnits, Knap.getValue)
    print('\nUse greedy by cost to allocate a weight of', maxUnits)
    testGreedy(inp, maxUnits,
               lambda x: 1/Knap.getCost(x))
    print('\nUse greedy by density to allocate a weight of', maxUnits)
    testGreedy(inp, maxUnits, Knap.density)

In [6]:
#list of inputs
names = ['book', 'pen', 'pencil', 'ipad', 'iphone',
         'headphones', 'flask', 'food', 'portable charger']
values = [50,10,5,70,90,79,50,80,70]
weight = [300,50,30,350,365,150,150,195,400]

inp = buildInput(names, values, weight)
#running the greedy algorithm 
testGreedys(inp, 800)

Use greedy by value to allocate a weight of 800
Total value of items taken = 264.0
    iphone: <90, 365>
    food: <80, 195>
    headphones: <79, 150>
    pen: <10, 50>
    pencil: <5, 30>

Use greedy by cost to allocate a weight of 800
Total value of items taken = 224.0
    pencil: <5, 30>
    pen: <10, 50>
    headphones: <79, 150>
    flask: <50, 150>
    food: <80, 195>

Use greedy by density to allocate a weight of 800
Total value of items taken = 224.0
    headphones: <79, 150>
    food: <80, 195>
    flask: <50, 150>
    pen: <10, 50>
    pencil: <5, 30>


Problems with greedy algorithms:

1. We get stuck in a local optima. Therefore we do not find the best answer. 
2. Due to 1. we do not even know how good our solution is.

Advantages:

1. nlogn complexity.

# Approach 2: Brute Force

Intution behind a brute force algorithm is that it will check every possible answer and then choose the possible answer.

S1: Search tree implementation

In [7]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #No need to explore left branch only look at right branch
        result = maxVal(toConsider[1:], avail) #recurssive implementation
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result


In [8]:
def testMaxVal(inp, maxUnits, printItems = True):
    print('Use search tree to allocate a weight of', maxUnits)
    val, taken = maxVal(inp, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

In [9]:
inp = buildInput(names, values, weight)
testMaxVal(inp, 800)

Use search tree to allocate a weight of 800
Total value of items taken = 264
    food: <80, 195>
    headphones: <79, 150>
    iphone: <90, 365>
    pencil: <5, 30>
    pen: <10, 50>


Problems with brute force:

1. TIME COMPLEXITY. For real world problems the search space is usually large and if we were to use search trees which have a complexity of 2^(n+1) our program will never stop running.

Advantages: 

1. Easy to implement and will find an optimal solution (if it exists).

# Approach 3: Dynamic Programming


In this case we will use memoization to check for a pre-existing solution that we computed earlier and only compute a new one if there is no previous solution. 

In [10]:
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #No need to explore left branch only look at right branch
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result



In [11]:
def testMaxVal(inp, maxUnits, algorithm, printItems = True):
    print('Input contains', len(inp), 'items')
    print('Use search tree to allocate a weight of', maxUnits)
    val, taken = algorithm(inp, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)

Building a bigger input list to show DP's use case

In [12]:
import random

def buildHugeInput(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Knap(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxCost)))
    return items

In [13]:
inp = buildHugeInput(50,1000,3000)
testMaxVal(inp, 800, fastMaxVal, True)

Input contains 50 items
Use search tree to allocate a weight of 800
Total value of items taken = 1345
    31: <367, 99>
    19: <978, 621>


Problems with DP:

1. Will only work when it's two criteria's of optimal substructure and overlapping subproblems are met.
2. For some problems if every option is a new answer then it is as good as brute force

Advatages:

1. When it's conditions are met, it can help decrease the run time drastically.
    